In [23]:
import re
import urllib.request
import pandas as pd
import urllib.request
from datetime import datetime
import os
from glob import glob

In [24]:
for i in range(1, 28):
    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1=2000&year2=2020&type=Mean"
    wp = urllib.request.urlopen(url)
    text = wp.read()

    dat = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    out = open("obl_{}_{}.csv".format(i, dat), 'wb')
    out.write(text)
    out.close()

Завантаження файлів та додання дати та часу до назви

In [25]:
def clean(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    start_index = 0
    for i, line in enumerate(lines):
        if '<tt><pre>' in line:
            lines[i] = line.replace('<tt><pre>', '')
            start_index = i
            break

    cleaned_lines = lines[start_index:-1]

    with open(path, 'w') as f:
        f.writelines(cleaned_lines)


def clean_all(folder):
    csv_files = glob('*.csv')

    for file in csv_files:
        clean(file)


clean_all(r"C:\Users\Defje\PycharmProjects\Lab2")

Видалення зайвих даних з файлу

In [26]:
def dataframe(path):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Region']
    pat = 'obl_*.csv'
    end = os.path.join(path, pat)

    csv_file = glob(end)
    final_df = pd.DataFrame()
    for file in csv_file:
        base = os.path.basename(file)
        df = pd.read_csv(file, names=headers)

        num = re.search(r'\d+', base)
        ind = int(num.group())

        df['Region'] = ind

        final_df = pd.concat([final_df, df], ignore_index=True)
    return final_df

Функція для створення одного датафрейму з усіх файлів

In [27]:
def replacement(df):
    df['Region'] = df['Region'].replace({
        1: 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21,
        11: 9, 12: 26, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16,
        20: 27, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5
    })
    df = df.drop(df.loc[df['VHI'] == -1].index)
    return df

Заміна номерів регіонів і відкидання усіх NaN

In [28]:
def vhi(df, region, year):
    columns = ['Year', 'Region', 'VHI']
    filtered = df[(df["Region"] == region) & (df["Year"] == year)]
    return filtered[columns]

Ряд VHI для області за вказаний рік

In [29]:
def ext(df, years, regions):
    filtered = df[df['Year'].isin(years) & df['Region'].isin(regions)]

    final = filtered.groupby(['Region', 'Year']).agg(
        MinVHI=pd.NamedAgg(column="VHI", aggfunc="min"),
        MaxVHI=pd.NamedAgg(column="VHI", aggfunc="max"),
        MeanVHI=pd.NamedAgg(column="VHI", aggfunc="mean"),
        MedianVHI=pd.NamedAgg(column="VHI", aggfunc="median"))
    return final

Пошук екстремумів (min та max) для вказаних областей та років, середнього, медіани

In [30]:
def vhi2(df, year1, year2, regions):
    columns = ['Year', 'Region', 'VHI']
    years = range(year1, year2)
    filtered = df[df['Year'].isin(years) & df['Region'].isin(regions)]
    return filtered[columns]

Ряд VHI за вказаний діапазон років для вказаних областей

In [31]:
def drought(df):
    columns = ['Year', 'Region', 'VHI']
    filtered = df[df['VHI'] < 15]
    unique_regions = filtered.groupby('Year')['Region'].nunique()
    final_years = unique_regions[unique_regions >= 5].index
    final_dataframe = filtered[filtered['Year'].isin(final_years)]

    return final_dataframe[columns]

Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [32]:
print(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2"))

       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      2000     1  0.033  260.29  21.28  40.64  30.96      10
1      2000     2  0.033  260.40  22.74  42.06  32.40      10
2      2000     3  0.036  261.40  28.74  39.77  34.26      10
3      2000     4  0.043  262.45  36.16  37.55  36.86      10
4      2000     5  0.049  264.25  39.26  35.74  37.50      10
...     ...   ...    ...     ...    ...    ...    ...     ...
29479  2020    48  0.159  277.21  66.52  16.25  41.37       9
29480  2020    49  0.150  275.06  65.68  19.44  42.55       9
29481  2020    50  0.144  274.00  66.17  18.69  42.43       9
29482  2020    51  0.138  272.73  66.79  19.59  43.19       9
29483  2020    52  0.132  271.44  67.24  21.90  44.57       9

[29484 rows x 8 columns]


 Виконання функції dataframe

In [33]:
print(replacement(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2")))

       Year  Week    SMN     SMT    VCI    TCI    VHI  Region
0      2000     1  0.033  260.29  21.28  40.64  30.96      21
1      2000     2  0.033  260.40  22.74  42.06  32.40      21
2      2000     3  0.036  261.40  28.74  39.77  34.26      21
3      2000     4  0.043  262.45  36.16  37.55  36.86      21
4      2000     5  0.049  264.25  39.26  35.74  37.50      21
...     ...   ...    ...     ...    ...    ...    ...     ...
29479  2020    48  0.159  277.21  66.52  16.25  41.37      20
29480  2020    49  0.150  275.06  65.68  19.44  42.55      20
29481  2020    50  0.144  274.00  66.17  18.69  42.43      20
29482  2020    51  0.138  272.73  66.79  19.59  43.19      20
29483  2020    52  0.132  271.44  67.24  21.90  44.57      20

[28944 rows x 8 columns]


Виконання функції replacement

In [34]:
print(vhi(replacement(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2")),11, 2001))

      Year  Region    VHI
4420  2001      11  39.81
4421  2001      11  41.92
4422  2001      11  45.18
4423  2001      11  44.31
4424  2001      11  41.63
4425  2001      11  41.66
4426  2001      11  41.99
4427  2001      11  41.30
4428  2001      11  40.92
4429  2001      11  42.74
4430  2001      11  43.66
4431  2001      11  43.46
4432  2001      11  42.84
4433  2001      11  43.07
4434  2001      11  42.97
4435  2001      11  44.25
4436  2001      11  49.09
4437  2001      11  57.47
4438  2001      11  65.81
4439  2001      11  70.58
4440  2001      11  73.22
4441  2001      11  75.34
4442  2001      11  76.29
4443  2001      11  76.77
4444  2001      11  75.90
4445  2001      11  71.54
4446  2001      11  64.59
4447  2001      11  59.10
4448  2001      11  55.17
4449  2001      11  51.62
4450  2001      11  49.76
4451  2001      11  50.52
4452  2001      11  50.58
4453  2001      11  50.78
4454  2001      11  54.54
4455  2001      11  58.06
4456  2001      11  60.26
4457  2001  

Виконання функції vhi

In [35]:
print(ext(replacement(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2")),[2000,2002,2005],[12,15,17,21]))

             MinVHI  MaxVHI    MeanVHI  MedianVHI
Region Year                                      
12     2000   29.53   69.96  46.678654     46.530
       2002   40.84   68.61  52.898269     53.670
       2005   43.52   61.93  53.447059     54.560
15     2000   22.52   68.24  45.402885     44.570
       2002   37.83   72.76  55.293269     52.925
       2005   37.50   66.14  51.529412     51.040
17     2000   16.81   66.21  44.855962     43.425
       2002   26.96   66.84  51.330962     52.710
       2005   35.16   66.48  52.957647     54.230
21     2000   20.97   65.00  43.941346     46.415
       2002   39.19   64.10  49.779038     48.970
       2005   44.19   62.26  53.556667     53.970


Виконання функції ext

In [36]:
print(vhi2(replacement(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2")),2000,2005,[12,15,17,21]))

       Year  Region    VHI
0      2000      21  30.96
1      2000      21  32.40
2      2000      21  34.26
3      2000      21  36.86
4      2000      21  37.50
...     ...     ...    ...
13359  2004      17  56.44
13360  2004      17  56.01
13361  2004      17  54.29
13362  2004      17  55.56
13363  2004      17  57.24

[964 rows x 3 columns]


Виконання функції vhi2

In [37]:
print(drought(replacement(dataframe(r"C:\Users\Defje\PycharmProjects\Lab2"))))

       Year  Region    VHI
1135   2000       9  12.51
1136   2000       9  10.60
1137   2000       9  11.20
1138   2000       9  12.32
1139   2000       9  14.65
...     ...     ...    ...
28784  2007      20  12.96
28785  2007      20  13.48
28786  2007      20  14.05
28787  2007      20  14.41
28788  2007      20  14.81

[88 rows x 3 columns]


Виконання функції drought

In [38]:
def delete():
    path = r"C:\Users\Defje\PycharmProjects\Lab2"
    files = glob(os.path.join(path, '*.csv'))
    for file_path in files:
        os.remove(file_path)


delete()

Видалення усіх файлів з росширенням .csv